---
---
# 🎥 Gerçek Zamanlı Webcam Inference
---
---

# Önce python modelini aşağıya bırakıyorum.Bundan önceki projede kullanılan aynı kodlar kullanılıcaktır.Önce .pth ve .pt dosyalarını oluşturalım.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# --------------------------
# Geliştirilmiş CNN Mimarisi
# --------------------------

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)


class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedCNN, self).__init__()

        self.features = nn.Sequential(
            # Giriş: [B, 3, 64, 64]
            ConvBlock(3, 32),              # [B, 32, 64, 64]
            ConvBlock(32, 64),             # [B, 64, 64, 64]
            nn.MaxPool2d(2, 2),            # [B, 64, 32, 32]

            ConvBlock(64, 128),            # [B, 128, 32, 32]
            ConvBlock(128, 128),           # [B, 128, 32, 32]
            nn.MaxPool2d(2, 2),            # [B, 128, 16, 16]

            ConvBlock(128, 256),           # [B, 256, 16, 16]
            nn.MaxPool2d(2, 2),            # [B, 256, 8, 8]

            # Global Average Pooling: [B, 256, 1, 1]
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                  # [B, 256]
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes)    # [B, num_classes]
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# --------------------------
# Veri hazırlığı
# --------------------------

# CIFAR-10 için tipik normalize değerleri (RGB)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 0-1
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    ),
])

train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

print("Dataset hazır, model oluşturuluyor...")

# --------------------------
# Model, cihaz, loss, optimizer
# --------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanilan cihaz:", device)

model = ImprovedCNN(num_classes=10).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# İstersen scheduler da ekleyebilirsin:
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

EPOCHS = 1  # İstersen artır (10-20 arası daha makul)


# --------------------------
# Eğitim + basit eval döngüsü
# --------------------------

best_acc = 0.0

for epoch in range(EPOCHS):
    # ---- Train ----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total

    # ---- Eval (test) ----
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = outputs.max(1)
            correct_test += preds.eq(labels).sum().item()
            total_test += labels.size(0)

    test_acc = 100.0 * correct_test / total_test

    # scheduler kullanıyorsan burada step:
    # scheduler.step()

    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.2f}% | "
          f"Test Acc: {test_acc:.2f}%")

    # En iyi modeli kaydet
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"  -> Yeni en iyi model kaydedildi! (best_model.pth, acc={best_acc:.2f}%)")

print("Egitim tamamlandi. En iyi test accuracy:", best_acc)


# --------------------------
# TorchScript export (LibTorch için)
# --------------------------

# Aynı mimariyi yeniden kur
export_model = ImprovedCNN(num_classes=10)
export_model.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
export_model.eval()

# Örnek giriş (dummy input)
example_input = torch.randn(1, 3, 64, 64)

# TorchScript trace
traced_script_module = torch.jit.trace(export_model, example_input)

# Kaydet
traced_script_module.save("model_ts.pt")

print("TorchScript modeli kaydedildi: model_ts.pt")

100%|██████████| 170M/170M [02:33<00:00, 1.11MB/s] 


Dataset hazır, model oluşturuluyor...
Kullanilan cihaz: cpu
ImprovedCNN(
  (features): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (4): ConvBlock(
 

-----

# C++ Kodunu yine aşağıya bırakıyorum:

```cpp
// ---------------------------------------------------------
// LibTorch + OpenCV ile Gerçek Zamanlı Webcam Inference
// (ImprovedCNN + CIFAR10 Normalize)
// ---------------------------------------------------------

#include <iostream>
#include <fstream>
#include <string>
#include <vector>

#include <torch/script.h>
#include <opencv2/opencv.hpp>

// CIFAR-10 sınıf isimleri
const std::vector<std::string> CIFAR10_CLASSES = {
    "airplane",   // 0
    "automobile", // 1
    "bird",       // 2
    "cat",        // 3
    "deer",       // 4
    "dog",        // 5
    "frog",       // 6
    "horse",      // 7
    "ship",       // 8
    "truck"       // 9
};

// CIFAR-10 Normalize değerleri (PyTorch ile aynı)
const float CIFAR10_MEAN[3] = {0.4914f, 0.4822f, 0.4465f}; // R, G, B
const float CIFAR10_STD[3] = {0.2470f, 0.2435f, 0.2616f};

torch::Tensor preprocess_image(const cv::Mat &img_bgr, int target_size)
{
    cv::Mat img;
    img_bgr.copyTo(img);

    // Resize
    cv::resize(img, img, cv::Size(target_size, target_size));

    // BGR -> RGB
    cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

    // uint8 -> float32 [0,1]
    cv::Mat img_float;
    img.convertTo(img_float, CV_32F, 1.0f / 255.0f);

    // Kanal bazlı normalize
    {
        std::vector<cv::Mat> channels(3);
        cv::split(img_float, channels); // R,G,B

        for (int c = 0; c < 3; ++c)
        {
            channels[c] = (channels[c] - CIFAR10_MEAN[c]) / CIFAR10_STD[c];
        }

        cv::merge(channels, img_float);
    }

    // HWC -> NHWC tensor
    auto tensor_image = torch::from_blob(
        img_float.data,
        {1, img_float.rows, img_float.cols, 3}, // [N,H,W,C]
        torch::TensorOptions().dtype(torch::kFloat32));

    // NHWC -> NCHW
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N,C,H,W]

    // Belleği sabitle
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}

torch::Tensor run_inference(torch::jit::script::Module &module,
                            const torch::Tensor &input)
{
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);
    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}

int main()
{
    try
    {
        std::cout << "[INFO] Program basladi.\n";

        // ------------------------------
        // 1) Modeli yükle
        // ------------------------------
        const std::string model_path = "model_ts.pt";

        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] model_ts.pt bulunamadi! Beklenen yer: "
                          << model_path << "\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }

        std::cout << "[INFO] Model yukleniyor...\n";
        torch::jit::script::Module module = torch::jit::load(model_path);
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yüklendi, CPU'da ve eval modunda.\n";

        // ------------------------------
        // 2) Webcam aç
        // ------------------------------
        cv::VideoCapture cap(0); // 0 = varsayilan kamera

        if (!cap.isOpened())
        {
            std::cerr << "[HATA] Kamera acilamadi!\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        std::cout << "[INFO] Webcam acildi. 'q' tusuna basarak cikabilirsin.\n";

        const int target_size = 64;

        while (true)
        {
            cv::Mat frame;
            cap >> frame; // kameradan bir frame oku

            if (frame.empty())
            {
                std::cerr << "[WARN] Bos frame geldi, devam ediyorum...\n";
                continue;
            }

            // ------------------------------
            // 3) Preprocess -> Tensor
            // ------------------------------
            torch::Tensor input_tensor = preprocess_image(frame, target_size);

            // ------------------------------
            // 4) Inference
            // ------------------------------
            torch::Tensor output = run_inference(module, input_tensor);

            // ------------------------------
            // 5) Postprocess: softmax + argmax
            // ------------------------------
            torch::Tensor probs = torch::softmax(output, 1);
            torch::Tensor pred_class = probs.argmax(1);

            int pred_idx = pred_class.item<int>();
            std::string pred_name = "unknown";

            if (pred_idx >= 0 && pred_idx < static_cast<int>(CIFAR10_CLASSES.size()))
            {
                pred_name = CIFAR10_CLASSES[pred_idx];
            }

            // ------------------------------
            // 6) Sonucu frame üzerine yaz
            // ------------------------------
            std::string text = "Pred: " + pred_name + " (idx=" + std::to_string(pred_idx) + ")";
            cv::putText(frame, text,
                        cv::Point(10, 30),
                        cv::FONT_HERSHEY_SIMPLEX,
                        0.8,
                        cv::Scalar(0, 255, 0),
                        2);

            // FPS vs. istersek buraya ekleyebiliriz.

            // Frame'i göster
            cv::imshow("Real-time CIFAR10 Inference", frame);

            // 'q' veya ESC ile cik
            char key = static_cast<char>(cv::waitKey(1));
            if (key == 'q' || key == 27)
            {
                std::cout << "[INFO] Kullanici cikis istegi verdi.\n";
                break;
            }
        }

        cap.release();
        cv::destroyAllWindows();

        std::cout << "[INFO] Program sonlandi. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error &e)
    {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception &e)
    {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}


-----

## .txt ve build işlemleri diğer projelerde olduğunu gibi devam edecektir...
----